In [1]:
import torch
import sys
from swift.llm import get_model_tokenizer
dirs = ["/home/hanlv/workspace/code/research/infodemic/LLM/swift/examples/pytorch/llm"]
for _dir in dirs:
    if _dir not in sys.path:
        sys.path.append(_dir)
from custom import CustomModelType, CustomTemplateType
model_type = CustomModelType.gemma_7b_it
model, tokenizer = get_model_tokenizer(
    model_type, 
    load_model=False
)


2024-04-20 23:13:48,154 - modelscope - INFO - PyTorch version 2.1.2 Found.
2024-04-20 23:13:48,157 - modelscope - INFO - Loading ast index from /home/hanlv/.cache/modelscope/ast_indexer
2024-04-20 23:13:48,185 - modelscope - INFO - Loading done! Current index file version is 1.13.3, with md5 2390e04ce69984a7eaf9207ea8d4380c and a total number of 972 components indexed
[INFO:swift] Loading the model using model_dir: /home/css/models/gemma-7b-it
[INFO:swift] Setting torch_dtype: torch.bfloat16
[INFO:swift] model_config: GemmaConfig {
  "_name_or_path": "/home/css/models/gemma-7b-it",
  "architectures": [
    "GemmaForCausalLM"
  ],
  "attention_bias": false,
  "attention_dropout": 0.0,
  "bos_token_id": 2,
  "eos_token_id": 1,
  "head_dim": 256,
  "hidden_act": "gelu",
  "hidden_activation": null,
  "hidden_size": 3072,
  "initializer_range": 0.02,
  "intermediate_size": 24576,
  "max_position_embeddings": 8192,
  "model_type": "gemma",
  "num_attention_heads": 16,
  "num_hidden_layers":

In [2]:
import jsonlines
from tqdm.notebook import tqdm

dict_list = []
version = "1"
with jsonlines.open(f"data{version}.jsonl", mode="r") as file_jsonl:
    for item in file_jsonl.iter():
        dict_list.append(item)

lens1 = []
lens2 = []

for item in tqdm(dict_list):
    lens1.append(len(tokenizer(item["query"])["input_ids"]))
    # lens2.append(len(tokenizer.tokenize(item["query"])))



  0%|          | 0/12192 [00:00<?, ?it/s]

In [3]:
# gemma 3468 = mixtral 3686
sorted(lens1, reverse=True)[0: 10]


[3457, 3363, 3359, 3291, 3230, 3225, 3212, 3208, 3199, 3199]

修改较长的句子

显卡 24GB

7B模型，长度控制在3600以下 


In [4]:
del_ids = []
for i, item in enumerate(lens1):
    if item == 3691:
        del_ids.append(i)

del_ids


[11650]

In [13]:
# print(len(tokenizer(dict_list[4157]["query"])["input_ids"]))
dict_list[1862]["query"]


"According to the CLAIM and the PRIOR KNOWLEDGE, please classify the CLAIM as TRUE or FALSE. If the content described by the CLAIM is correct, then classify it as TRUE; if the content described by the CLAIM is incorrect, then classify it as FALSE.\n\nCLAIM:\nPublication date: 2020-02-20\nContent: Social media posts claim Chloroquine, an antimalarial drug, can cure COVID-19.\n\nPRIOR KNOWLEDGE:\nInformation Expansion and Summary:\n\nInformation 1 describes the FDA's review of safety issues related to the use of hydroxychloroquine and chloroquine to treat hospitalized patients with COVID-19. The FDA has revoked the emergency use authorization (EUA) for these drugs in treating COVID-19 due to serious heart rhythm problems and other safety issues, including blood and lymph system disorders, kidney injuries, and liver problems and failure. The FDA also warns of increased use of these medicines through outpatient prescriptions and reports of serious heart rhythm problems in patients with COV

In [5]:

print(dict_list[11650]["query"])


According to the CLAIM and the PRIOR KNOWLEDGE, please classify the CLAIM as TRUE or FALSE. If the content described by the CLAIM is correct, then classify it as TRUE; if the content described by the CLAIM is incorrect, then classify it as FALSE.

CLAIM:
Publication date: 2020-04-27
Content: A 2005 NIH study shows that Fauci already knew of chloroquine’s effectiveness against coronaviruses, including SARS-CoV-2.

PRIOR KNOWLEDGE:
Information 1:
Publication date: None
Title: Chloroquine is a potent inhibitor of SARS coronavirus infection ...
Content:
Copyright © 2005 Vincent et al; licensee BioMed Central Ltd. This is an Open Access article distributed under the terms of the Creative Commons Attribution License (http://creativecommons.org/licenses/by/2.0), which permits unrestricted use, distribution, and reproduction in any medium, provided the original work is properly cited. Severe acute respiratory syndrome (SARS) is caused by a newly discovered coronavirus (SARS-CoV).
NH4Cl exhibit